In [8]:
# Import dependencies

import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt


In [9]:
# Import .CSV files
df_ca2019 = pd.read_csv(r"data/Los Angeles 2019 data.csv")
df_ca2020 = pd.read_csv(r"data/Los Angeles 2020 data.csv")
df_ca2021 = pd.read_csv(r"data/Los Angeles 2021 data.csv")
df_ak2019 = pd.read_csv(r"data/Juneau 2019 data.csv")
df_ak2020=pd.read_csv(r"data/Juneau 2020 data.csv")
df_ak2021=pd.read_csv(r"data/Juneau 2021 data.csv")
df_fl2019 = pd.read_csv(r"data/Tallahassee 2019 data.csv")
df_fl2020=pd.read_csv(r"data/Tallahassee 2020 data.csv")
df_fl2021=pd.read_csv(r"data/Tallahassee 2021 data.csv")


In [16]:
# Parse LA Data
#2019
df_ca2019_parsed = df_ca2019[['Date', 'CBSA Name','Local Site Name', 'Daily AQI Value', 'Daily Mean PM2.5 Concentration']].copy()
df_ca2019_parsed['Date'] = pd.to_datetime(df_ca2019_parsed['Date'])
df_ca2019_parsed.set_index('Date', inplace=True)
df_ca2019_parsed.dropna(inplace=True)
df_ca2019_cleaned = df_ca2019_parsed[df_ca2019_parsed['Local Site Name'] == 'Compton']


#2020
df_ca2020_parsed =df_ca2020[['Date','CBSA Name','Local Site Name', 'Daily AQI Value', 'Daily Mean PM2.5 Concentration']].copy()
df_ca2020_parsed['Date'] = pd.to_datetime(df_ca2020_parsed['Date'])
df_ca2020_parsed.set_index('Date', inplace=True)
df_ca2020_parsed.dropna(inplace=True)
df_ca2020_cleaned= df_ca2020_parsed[df_ca2020_parsed['Local Site Name'] == 'Compton']

#2021
df_ca2021_parsed =df_ca2021[['Date','CBSA Name','Local Site Name', 'Daily AQI Value', 'Daily Mean PM2.5 Concentration']].copy()
df_ca2021_parsed['Date'] = pd.to_datetime(df_ca2021_parsed['Date'])
df_ca2021_parsed.set_index('Date', inplace=True)
df_ca2021_parsed.dropna(inplace=True)
df_ca2021_cleaned= df_ca2021_parsed[df_ca2021_parsed['Local Site Name'] == 'Compton']

#Concat to one dataframe
df_cacombined = pd.concat([df_ca2019_cleaned, 
                              df_ca2020_cleaned, 
                              df_ca2021_cleaned], 
                             axis=0)


df_cacombined





,CBSA Name,Local Site Name,Daily AQI Value,Daily Mean PM2.5 Concentration
Date,,,,
2019-01-03,"Los Angeles-Long Beach-Anaheim, CA",Compton,57,12.1
2019-01-09,"Los Angeles-Long Beach-Anaheim, CA",Compton,77,23.0
2019-01-12,"Los Angeles-Long Beach-Anaheim, CA",Compton,55,11.1
2019-01-15,"Los Angeles-Long Beach-Anaheim, CA",Compton,41,7.3
2019-01-18,"Los Angeles-Long Beach-Anaheim, CA",Compton,56,11.9
...,...,...,...,...
2021-12-27,"Los Angeles-Long Beach-Anaheim, CA",Compton,51,9.3
2021-12-28,"Los Angeles-Long Beach-Anaheim, CA",Compton,51,9.2
2021-12-29,"Los Angeles-Long Beach-Anaheim, CA",Compton,41,7.4


In [18]:
#Parse Tallahasse Data
#2019
df_fl2019_parsed = df_fl2019[['Date', 'CBSA Name','Local Site Name', 'Daily AQI Value', 'Daily Mean PM2.5 Concentration']].copy()
df_fl2019_parsed['Date'] = pd.to_datetime(df_fl2019_parsed['Date'])
df_fl2019_parsed.set_index('Date', inplace=True)
df_fl2019_parsed.dropna(inplace=True)
df_fl2019_cleaned = df_fl2019_parsed[df_fl2019_parsed['Local Site Name'] == 'Tallahassee Community College']

#2020
df_fl2020_parsed = df_fl2020[['Date', 'CBSA Name','Local Site Name', 'Daily AQI Value', 'Daily Mean PM2.5 Concentration']].copy()
df_fl2020_parsed['Date'] = pd.to_datetime(df_fl2020_parsed['Date'])
df_fl2020_parsed.set_index('Date', inplace=True)
df_fl2020_parsed.dropna(inplace=True)
df_fl2020_cleaned = df_fl2020_parsed[df_fl2020_parsed['Local Site Name'] == 'Tallahassee Community College']

#2021
df_fl2021_parsed = df_fl2021[['Date', 'CBSA Name','Local Site Name', 'Daily AQI Value', 'Daily Mean PM2.5 Concentration']].copy()
df_fl2021_parsed['Date'] = pd.to_datetime(df_fl2021_parsed['Date'])
df_fl2021_parsed.set_index('Date', inplace=True)
df_fl2021_parsed.dropna(inplace=True)
df_fl2021_cleaned = df_fl2021_parsed[df_fl2021_parsed['Local Site Name'] == 'Tallahassee Community College']

#Concat to one dataframe
df_flcombined = pd.concat([df_fl2019_cleaned, 
                              df_fl2020_cleaned, 
                              df_fl2021_cleaned], 
                             axis=0)


df_flcombined


,CBSA Name,Local Site Name,Daily AQI Value,Daily Mean PM2.5 Concentration
Date,,,,
2019-01-03,"Tallahassee, FL",Tallahassee Community College,17,3.0
2019-01-06,"Tallahassee, FL",Tallahassee Community College,37,6.6
2019-01-09,"Tallahassee, FL",Tallahassee Community College,22,4.0
2019-01-12,"Tallahassee, FL",Tallahassee Community College,24,4.4
2019-01-15,"Tallahassee, FL",Tallahassee Community College,43,7.7
...,...,...,...,...
2021-12-27,"Tallahassee, FL",Tallahassee Community College,27,4.8
2021-12-28,"Tallahassee, FL",Tallahassee Community College,27,4.9
2021-12-29,"Tallahassee, FL",Tallahassee Community College,45,8.1


In [19]:
#Alaska Data
# 2019 Data:
df_ak2019_parsed =df_ak2019[['Date', 'CBSA Name','Local Site Name', 'Daily AQI Value', 'Daily Mean PM2.5 Concentration']].copy()
df_ak2019_parsed['Date'] = pd.to_datetime(df_ak2019_parsed['Date'])
df_ak2019_parsed.set_index('Date', inplace=True)
df_ak2019_parsed.dropna(inplace=True)
df_ak2019_parsed
# 2020 Data:
df_ak2020_parsed =df_ak2020[['Date', 'CBSA Name','Local Site Name', 'Daily AQI Value', 'Daily Mean PM2.5 Concentration']].copy()
df_ak2020_parsed['Date'] = pd.to_datetime(df_ak2020_parsed['Date'])
df_ak2020_parsed.set_index('Date', inplace=True)
df_ak2020_parsed.dropna(inplace=True)
df_ak2020_parsed
# 2021 Data
df_ak2021_parsed =df_ak2021[['Date', 'CBSA Name','Local Site Name', 'Daily AQI Value', 'Daily Mean PM2.5 Concentration']].copy()
df_ak2021_parsed['Date'] = pd.to_datetime(df_ak2021_parsed['Date'])
df_ak2021_parsed.set_index('Date', inplace=True)
df_ak2021_parsed.dropna(inplace=True)
df_ak2021_parsed
df_cacombined_ak= pd.concat([df_ak2019_parsed,
                         df_ak2020_parsed,
                         df_ak2021_parsed],
                         axis=0)
df_cacombined_ak

,CBSA Name,Local Site Name,Daily AQI Value,Daily Mean PM2.5 Concentration
Date,,,,
2019-01-01,"Juneau, AK",Floyd Dryden Middle School,39,7.0
2019-01-02,"Juneau, AK",Floyd Dryden Middle School,29,5.3
2019-01-03,"Juneau, AK",Floyd Dryden Middle School,57,12.2
2019-01-04,"Juneau, AK",Floyd Dryden Middle School,75,22.0
2019-01-05,"Juneau, AK",Floyd Dryden Middle School,81,25.4
...,...,...,...,...
2021-12-27,"Juneau, AK",Floyd Dryden Middle School,56,11.7
2021-12-28,"Juneau, AK",Floyd Dryden Middle School,62,15.2
2021-12-29,"Juneau, AK",Floyd Dryden Middle School,38,6.9
